<a href="https://colab.research.google.com/github/sooonsyk/Pocari/blob/main/classification/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **base model**


In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# 이미지 전처리 및 데이터 증강
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
from torchvision import models  #ResNet 사용

# 전이 학습에 사용할 사전 학습된 ResNet 모델 로드
model = models.resnet18(pretrained=True)

# 마지막 fully connected layer를 새로운 출력에 맞게 수정
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))  # dataset.classes는 이미지 데이터셋의 클래스 수

# GPU를 사용할 수 있는 경우 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct / total

        # 검증 데이터셋에 대한 성능 측정
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_accuracy = 100. * correct / total

        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

# 학습 및 평가 실행
train(model, criterion, optimizer, train_loader, val_loader)


Epoch [1/10], Train Loss: 1.9960, Train Accuracy: 25.93%, Val Loss: 2.5874, Val Accuracy: 0.00%
Epoch [2/10], Train Loss: 1.9073, Train Accuracy: 29.63%, Val Loss: 1.8904, Val Accuracy: 30.00%
Epoch [3/10], Train Loss: 2.3253, Train Accuracy: 37.04%, Val Loss: 1.3020, Val Accuracy: 60.00%
Epoch [4/10], Train Loss: 1.8678, Train Accuracy: 29.63%, Val Loss: 2.2113, Val Accuracy: 60.00%
Epoch [5/10], Train Loss: 2.2959, Train Accuracy: 25.93%, Val Loss: 1.2031, Val Accuracy: 60.00%
Epoch [6/10], Train Loss: 1.6042, Train Accuracy: 51.85%, Val Loss: 1.2877, Val Accuracy: 70.00%
Epoch [7/10], Train Loss: 1.8625, Train Accuracy: 40.74%, Val Loss: 2.6208, Val Accuracy: 10.00%
Epoch [8/10], Train Loss: 1.1540, Train Accuracy: 51.85%, Val Loss: 1.4108, Val Accuracy: 40.00%
Epoch [9/10], Train Loss: 1.1536, Train Accuracy: 66.67%, Val Loss: 1.3695, Val Accuracy: 40.00%
Epoch [10/10], Train Loss: 0.9265, Train Accuracy: 70.37%, Val Loss: 1.9548, Val Accuracy: 20.00%


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 10.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [2, 2, 0, 0, 2, 2, 3, 0, 2, 3]


### **white balance 추가**


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte                        #이미지 처리 - white balance 에 활용
from PIL import Image

def white_patch(image, percentile=100):

    """
    Parameters
    ----------
    image : 입력으로 받는 이미지는 RGB 채널을 각각 가지는 (height, width, channels) 3차원 numpy 배열
    percentile : 채널 값의 보정값으로 사용하고자 하는 값을 지정하기 위한 비율, 기본값은 100으로 최대값을 사용
    """

    image = np.array(image)

    white_patch_image = img_as_ubyte(
        (image * 1.0 / np.percentile(image,
                                     percentile,
                                     axis=(0, 1))).clip(0, 1))
    return Image.fromarray(white_patch_image)

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Lambda(lambda x: white_patch(x, percentile=85)),  # 전처리 함수 적용
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
from torchvision import models  #ResNet 사용

# 전이 학습에 사용할 사전 학습된 ResNet 모델 로드
model = models.resnet18(pretrained=True)

# 마지막 fully connected layer를 새로운 출력에 맞게 수정
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))  # dataset.classes는 이미지 데이터셋의 클래스 수

# GPU를 사용할 수 있는 경우 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct / total

        # 검증 데이터셋에 대한 성능 측정
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_accuracy = 100. * correct / total

        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

# 학습 및 평가 실행
train(model, criterion, optimizer, train_loader, val_loader)


Epoch [1/10], Train Loss: 1.9344, Train Accuracy: 25.93%, Val Loss: 1.6765, Val Accuracy: 10.00%
Epoch [2/10], Train Loss: 2.1372, Train Accuracy: 11.11%, Val Loss: 1.8797, Val Accuracy: 60.00%
Epoch [3/10], Train Loss: 1.4425, Train Accuracy: 40.74%, Val Loss: 1.9368, Val Accuracy: 30.00%
Epoch [4/10], Train Loss: 2.2977, Train Accuracy: 44.44%, Val Loss: 2.4211, Val Accuracy: 10.00%
Epoch [5/10], Train Loss: 1.8399, Train Accuracy: 37.04%, Val Loss: 1.9213, Val Accuracy: 30.00%
Epoch [6/10], Train Loss: 1.6175, Train Accuracy: 44.44%, Val Loss: 1.6453, Val Accuracy: 40.00%
Epoch [7/10], Train Loss: 1.6030, Train Accuracy: 48.15%, Val Loss: 2.7516, Val Accuracy: 0.00%
Epoch [8/10], Train Loss: 2.1997, Train Accuracy: 25.93%, Val Loss: 2.6642, Val Accuracy: 20.00%
Epoch [9/10], Train Loss: 1.6983, Train Accuracy: 44.44%, Val Loss: 1.7541, Val Accuracy: 30.00%
Epoch [10/10], Train Loss: 1.0680, Train Accuracy: 59.26%, Val Loss: 2.6596, Val Accuracy: 10.00%


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 10.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [2, 3, 3, 0, 2, 1, 0, 2, 0, 2]


### **이미지 증강 추가**


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from skimage import img_as_ubyte                        #이미지 처리 - white balance 에 활용
from PIL import Image

def white_patch(image, percentile=100):

    """
    Parameters
    ----------
    image : 입력으로 받는 이미지는 RGB 채널을 각각 가지는 (height, width, channels) 3차원 numpy 배열
    percentile : 채널 값의 보정값으로 사용하고자 하는 값을 지정하기 위한 비율, 기본값은 100으로 최대값을 사용
    """

    image = np.array(image)

    white_patch_image = img_as_ubyte(
        (image * 1.0 / np.percentile(image,
                                     percentile,
                                     axis=(0, 1))).clip(0, 1))
    return Image.fromarray(white_patch_image)


transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.Lambda(lambda x: white_patch(x, percentile=85)),  # 전처리 함수 적용
    transforms.RandomHorizontalFlip(),  # 랜덤으로 이미지를 수평으로 뒤집기
    transforms.RandomVerticalFlip(p=0.5), # 랜덤으로 이미지를 수직으로 뒤집기
    transforms.RandomRotation(degrees=(-10, 10)), # 랜덤으로 이미지 회전
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 정규화
])

# 이미지 데이터셋 로드
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/images_0601', transform=transform)

# 데이터셋을 학습, 검증, 테스트로 분할
train_val_indices, test_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.25, random_state=42)  # train 60%, val 20%, test 20%

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

# DataLoader 생성 - data 수 적어서 batch 지정 안 함
train_loader = DataLoader(dataset=train_dataset, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, shuffle=False)

# 데이터셋 클래스 확인
print("Classes:", dataset.classes)
print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

Classes: ['blood', 'diarrhea', 'green', 'normal', 'white']
Number of training samples: 27
Number of validation samples: 10
Number of test samples: 10


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# 전이 학습에 사용할 사전 학습된 ResNet 모델 로드
model = models.resnet18(pretrained=True)

# 마지막 fully connected layer를 새로운 출력에 맞게 수정
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(dataset.classes))  # dataset.classes는 이미지 데이터셋의 클래스 수

# GPU를 사용할 수 있는 경우 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 학습 함수 정의
def train(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct / total

        # 검증 데이터셋에 대한 성능 측정
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_loss = val_loss / len(val_loader)
        val_accuracy = 100. * correct / total

        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

# 학습 및 평가 실행
train(model, criterion, optimizer, train_loader, val_loader)


Epoch [1/10], Train Loss: 2.6882, Train Accuracy: 22.22%, Val Loss: 1.5937, Val Accuracy: 60.00%
Epoch [2/10], Train Loss: 2.5051, Train Accuracy: 18.52%, Val Loss: 1.2505, Val Accuracy: 30.00%
Epoch [3/10], Train Loss: 3.2872, Train Accuracy: 11.11%, Val Loss: 1.5582, Val Accuracy: 30.00%
Epoch [4/10], Train Loss: 2.2890, Train Accuracy: 25.93%, Val Loss: 3.0036, Val Accuracy: 0.00%
Epoch [5/10], Train Loss: 3.4100, Train Accuracy: 22.22%, Val Loss: 2.8324, Val Accuracy: 0.00%
Epoch [6/10], Train Loss: 2.6505, Train Accuracy: 40.74%, Val Loss: 2.8419, Val Accuracy: 0.00%
Epoch [7/10], Train Loss: 2.2295, Train Accuracy: 40.74%, Val Loss: 2.3261, Val Accuracy: 10.00%
Epoch [8/10], Train Loss: 1.7492, Train Accuracy: 37.04%, Val Loss: 0.9496, Val Accuracy: 60.00%
Epoch [9/10], Train Loss: 1.9807, Train Accuracy: 29.63%, Val Loss: 1.4008, Val Accuracy: 40.00%
Epoch [10/10], Train Loss: 2.2895, Train Accuracy: 11.11%, Val Loss: 1.0019, Val Accuracy: 60.00%


In [ ]:
# 테스트 데이터셋을 사용하여 모델 평가
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    predicted_labels = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            # 예측된 레이블과 실제 레이블 저장
            predicted_labels.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = correct / total
    print(f'Test Accuracy: {test_accuracy:.2%}')
    return true_labels, predicted_labels

true_labels, predicted_labels = test(model, test_loader)

# 실제 레이블과 예측된 레이블 출력
print("True Labels:", true_labels)
print("Predicted Labels:", predicted_labels)

Test Accuracy: 20.00%
True Labels: [3, 4, 3, 4, 3, 4, 2, 3, 1, 3]
Predicted Labels: [3, 3, 4, 3, 4, 3, 4, 4, 3, 3]
